# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3 
conn = sqlite3.connect('school.sqlite')
cur = conn.cursor()


In [ ]:
# Create the database school.sqlite 


## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [2]:
# Your code here
cur.execute("""CREATE TABLE contactInfo (
                id SERIAL PRIMARY KEY,
                first_name VARCHAR(50),
                last_name VARCHAR(50),
                role VARCHAR(20),
                telephone_number VARCHAR(15),
                street VARCHAR(100),
                city VARCHAR(50),
                state VARCHAR(50),
                zipcode VARCHAR(10))
                """)

OperationalError: table contactInfo already exists

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [7]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [10]:
# Iterate over the contact list and populate the contactInfo table here
# Assuming `contacts` is a list of dictionaries
for contact in contacts:
    # Extracting individual values from the contact dictionary
    first_name = contact.get('first_name')
    last_name = contact.get('last_name')
    role = contact.get('role')
    telephone_number = contact.get('telephone_number')
    street = contact.get('street')
    city = contact.get('city')
    state = contact.get('state')
    zipcode = contact.get('zipcode')

    # Insert each contact's details into the contactInfo table
    cur.execute("""
        INSERT INTO contactInfo (first_name, last_name, role, telephone_number, street, city, state, zipcode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
        (first_name, last_name, role, telephone_number, street, city, state, zipcode)
    )

# Commit the changes to the database
conn.commit()



**Query the Table to Ensure it is populated**

In [ ]:
# Your code here 


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [11]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [16]:
# Create the grades table
cur.execute("""
    CREATE TABLE grades (
        userId INTEGER NOT NULL,
        courseId INTEGER NOT NULL,
        grade VARCHAR(2),
        PRIMARY KEY(userId, courseId)
    )
""")


OperationalError: table grades already exists

In [17]:
conn.commit()

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [22]:
# Find the duplicate entry
cur.execute("""
    SELECT first_name, last_name, role, telephone_number, COUNT(*)
    FROM contactInfo
    GROUP BY first_name, last_name, role, telephone_number
    HAVING COUNT(*) > 1;
""")
duplicates = cur.fetchall()
print(duplicates)

[(None, None, 'staff', None, 2), (None, None, 'student', None, 6)]


In [23]:
# Delete the duplicate entry
cur.execute("""
    DELETE FROM contactInfo
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM contactInfo
        GROUP BY first_name, last_name, role, telephone_number
    );
""")
conn.commit()

In [24]:
# Check that the duplicate entry was removed
cur.execute("""
    SELECT first_name, last_name, role, telephone_number, COUNT(*)
    FROM contactInfo
    GROUP BY first_name, last_name, role, telephone_number
    HAVING COUNT(*) > 1;
""")
no_duplicates = cur.fetchall()
print(no_duplicates)  # This should return an empty list if all duplicates are removed

[(None, None, 'staff', None, 2), (None, None, 'student', None, 6)]


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [26]:
# Update Ed's address
# Update Ed Lyman's address
cur.execute("""
    UPDATE contactInfo
    SET street = '2910 Simpson Avenue',
        city = 'York',
        state = 'PA',
        zipcode = '17403'
    WHERE first_name = 'Ed' AND last_name = 'Lyman';
""")

# Commit the changes to the database
conn.commit()


In [27]:
# Query the database to ensure the change was made
# Query the database to check Ed Lyman's updated address
cur.execute("""
    SELECT *
    FROM contactInfo
    WHERE first_name = 'Ed' AND last_name = 'Lyman';
""")

# Fetch the result
ed_info = cur.fetchall()

# Print the result to verify the update
print(ed_info)


[]


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [28]:
# Your code here
conn.commit()


## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 